In [1]:
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

In [2]:
#coding=utf-8
# import os
import re
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

from pyecharts import Overlap, Bar, Line, Gauge, Pie, Grid

from datetime import date, timedelta

from pyecharts import online
online()

# os.chdir('D:\OneDrive\python\data')

with pd.ExcelFile('D:/OneDrive/python/data/日报.xlsx') as xlsx:
    df_zhuce = pd.read_excel(xlsx,'注册')
    df_kaitong = pd.read_excel(xlsx,'开通')
    df_touzi = pd.read_excel(xlsx,'投资')
    df_quan = pd.read_excel(xlsx,'券')
    df_huikuan = pd.read_excel(xlsx,'回款')

# 一、注册统计

### 1.注册人数

In [3]:
df_zhuce['注册日期'] = pd.to_datetime(df_zhuce['注册时间']).dt.strftime('%m-%d')

zhuce_num = df_zhuce.groupby('注册日期')['用户名'].count()

yesterday = date.today() - timedelta(1)

print('截至 {0} ,本月累计注册人数 {1} 人。'.format(yesterday.strftime('%m-%d'),df_zhuce.shape[0]))

截至 09-04 ,本月累计注册人数 585 人。


In [4]:
attr = zhuce_num.index
v1 = zhuce_num.values
bar = Bar("每日注册","2018-08",width=900)
bar.add('注册人数',attr,v1,mark_line=["average"],mark_point=["max", "min"])
bar

### 2、各邀请渠道注册人数 

In [5]:
s_yw = df_zhuce[df_zhuce['邀请渠道'] == '业务邀请'].groupby('注册日期')['用户名'].count()
s_pt = df_zhuce[df_zhuce['邀请渠道'] == '普通邀请'].groupby('注册日期')['用户名'].count()
s_zz = df_zhuce[df_zhuce['邀请渠道'] == '自主注册'].groupby('注册日期')['用户名'].count()

df_yw = pd.DataFrame(s_yw,index=zhuce_num.index).fillna(0)
df_pt = pd.DataFrame(s_pt,index=zhuce_num.index).fillna(0)
df_zz = pd.DataFrame(s_zz,index=zhuce_num.index).fillna(0)

print('截至 {0} :\n普通邀请注册 {1} 人，\n业务邀请注册 {2} 人，\n自主注册（含推广）注册 {3} 人。'
      .format(yesterday.strftime('%m-%d'),
              df_pt['用户名'].sum(),
              df_yw['用户名'].sum(),
              df_zz['用户名'].sum()))

截至 09-04 :
普通邀请注册 3.0 人，
业务邀请注册 519 人，
自主注册（含推广）注册 63 人。


In [6]:
attr = zhuce_num.index
v1 = df_yw['用户名']
v2 = df_pt['用户名']
v3 = df_zz['用户名']

bar = Bar("每日注册","按邀请渠道统计",width=900)
bar.add('业务邀请',attr,v1,mark_line=["average"],mark_point=["max", "min"])
bar.add('普通邀请',attr,v2,mark_line=["average"],mark_point=["max", "min"])
bar.add('自主注册',attr,v3,mark_line=["average"],mark_point=["max", "min"])
bar

### 3、新注册 — 开通 — 投资转化率

In [7]:
df_touzi_new = df_touzi[df_touzi['注册时间'].str.contains('2018-08')]
new_percntage = df_touzi_new['用户名'].nunique() / df_kaitong.shape[0] * 100

tip = '本月注册 ' + str(df_zhuce.shape[0]) + ' 人，开通存管 ' + str(df_kaitong.shape[0]) + ' 人，投资 ' + str(df_touzi_new['用户名'].nunique()) + ' 人'

gauge = Gauge("投资-存管开通人数比例")
gauge.add(tip, "转化率", round(new_percntage))
gauge

### 4、注册用户投资统计

###### 4.1、投资金额和人数

In [8]:
df_touzi_new = df_touzi_new.copy()
df_touzi_new['投资日期'] = pd.to_datetime(df_touzi_new['投资时间']).dt.strftime('%m-%d')

gp_touzi_new = df_touzi_new.groupby(['投资日期','用户名'],as_index=False)['总投资金额'].sum()

gp_touzi_new_renshu = gp_touzi_new.groupby('投资日期')['用户名'].count()

gp_touzi_new_money = gp_touzi_new.groupby('投资日期')['总投资金额'].sum()

print('截至 {0} ,本月新注册用户累计投资额 {1} 元。'.format(yesterday.strftime('%m-%d'),gp_touzi_new_money.sum()))

截至 09-04 ,本月新注册用户累计投资额 52700.0 元。


In [9]:
attr = gp_touzi_new_renshu.index
value_renshu = gp_touzi_new_renshu.values
value_money = gp_touzi_new_money.values

bar = Bar(title="新注册用户投资金额、投资人数")
bar.add(
    "投资额",
    attr,
    value_money,
    yaxis_formatter=" 元",
)

line = Line()
line.add("投资人数", attr, value_renshu, yaxis_formatter=" 人")

overlap = Overlap(width=900)
overlap.add(bar)
overlap.add(line, is_add_yaxis=True, yaxis_index=1)
overlap

###### 4.2、投资标期限

In [10]:
gp_touzi_new_qixian = df_touzi_new.groupby(['计划期限'])['总投资金额'].sum()

print('截至 {0} :\n1月标累计投资额  {1} 元，\n3月标累计投资额  {2} 元，\n6月标累计投资额  {3} 元，\n12月标累计投资额 {4} 元。'
      .format(yesterday.strftime('%m-%d'),
              gp_touzi_new_qixian.loc['1月'],
              gp_touzi_new_qixian.loc['3月'],
              gp_touzi_new_qixian.loc['6月'],
              gp_touzi_new_qixian.loc['12月']))

截至 09-04 :
1月标累计投资额  500.0 元，
3月标累计投资额  6000.0 元，
6月标累计投资额  14200.0 元，
12月标累计投资额 32000.0 元。


In [11]:
attr_pie = gp_touzi_new_qixian.index
value_pie = gp_touzi_new_qixian.values
pie = Pie("新注册用户 当月投资期限 比例", title_pos='center', width=900)
pie.add(
    "投资额",
    attr_pie,
    value_pie,
    radius=[30, 75],
    rosetype="area",
    is_legend_show=False,
    is_label_show=True,
)
pie

# 二、投资统计

### 1、每日投资统计

In [12]:
df_touzi = df_touzi.copy()
df_touzi['投资日期'] = pd.to_datetime(df_touzi['投资时间']).dt.strftime('%m-%d')

gp_touzi_money = df_touzi.groupby('投资日期')['总投资金额'].sum()

df_touzi_pp = df_touzi.drop_duplicates(subset=['用户名','投资日期'])

gp_touzi_pp = df_touzi_pp.groupby('投资日期')['用户名'].count()

print('截至 {0} ,本月累计投资金额 {1} 元，累计投资 {2} 人。'
      .format(yesterday.strftime('%m-%d'),
              gp_touzi_money.sum(),
              df_touzi['用户名'].nunique()))

截至 09-04 ,本月累计投资金额 8015500.0 元，累计投资 573 人。


In [13]:
# from pyecharts import Overlap, Bar, Line

attr = gp_touzi_money.index
value_pp = gp_touzi_pp.values
value_money = np.around(gp_touzi_money.values / 10000)

bar = Bar(title="每日投资")
bar.add(
    "投资额",
    attr,
    value_money,
    yaxis_formatter=" 万元",
    mark_line=["average"],
    mark_point=["max", "min"]
)

line = Line()
line.add("投资人数", 
         attr, 
         value_pp,
         yaxis_formatter=" 人",
         mark_point=["max", "min"])

overlap = Overlap(width=900)
overlap.add(bar)
overlap.add(line, is_add_yaxis=True, yaxis_index=1)
overlap

### 2、各期限标投资统计

In [14]:
gp_touzi_1 = df_touzi[df_touzi['计划期限'] == '1月'].groupby('投资日期')['总投资金额'].sum()
gp_touzi_3 = df_touzi[df_touzi['计划期限'] == '3月'].groupby('投资日期')['总投资金额'].sum()
gp_touzi_6 = df_touzi[df_touzi['计划期限'] == '6月'].groupby('投资日期')['总投资金额'].sum()
gp_touzi_12 = df_touzi[df_touzi['计划期限'] == '12月'].groupby('投资日期')['总投资金额'].sum()

df_touzi_1 = pd.DataFrame(gp_touzi_1.values,index=zhuce_num.index,columns=['投资额']).fillna(0)
df_touzi_3 = pd.DataFrame(gp_touzi_3.values,index=zhuce_num.index,columns=['投资额']).fillna(0)
df_touzi_6 = pd.DataFrame(gp_touzi_6.values,index=zhuce_num.index,columns=['投资额']).fillna(0)
df_touzi_12 = pd.DataFrame(gp_touzi_12.values,index=zhuce_num.index,columns=['投资额']).fillna(0)

print('截至 {0} :\n1月标累计投资额  {1} 元，\n3月标累计投资额  {2} 元，\n6月标累计投资额  {3} 元，\n12月标累计投资额 {4} 元。'
      .format(yesterday.strftime('%m-%d'),
              df_touzi_1['投资额'].sum(),
              df_touzi_3['投资额'].sum(),
              df_touzi_6['投资额'].sum(),
              df_touzi_12['投资额'].sum()))

截至 09-04 :
1月标累计投资额  2455100.0 元，
3月标累计投资额  1782900.0 元，
6月标累计投资额  1426500.0 元，
12月标累计投资额 2351000.0 元。


In [15]:
# from pyecharts import Bar

attr = zhuce_num.index
v1 = df_touzi_1['投资额'].values
v2 = df_touzi_3['投资额'].values
v3 = df_touzi_6['投资额'].values
v4 = df_touzi_12['投资额'].values

bar = Bar("各期限标投资额",width=900)
bar.add("1月标", attr, v1, is_stack=True)
bar.add("3月标", attr, v2, is_stack=True)
bar.add("6月标", attr, v3, is_stack=True)
bar.add("12月标", attr, v4, is_stack=True)

bar

### 3、本月VIP投资统计

In [16]:
gp_touzi_vip = df_touzi.groupby('投资时VIP等级')['总投资金额'].sum()

# from pyecharts import Grid

attr_vip = gp_touzi_vip.index
value_vip = np.around(gp_touzi_vip.values / 10000)

bar = Bar("VIP投资额", width=900)
bar.add("投资额",
        attr_vip,
        value_vip,
        mark_point=["min", "max"],
        yaxis_formatter=" 万元",
        is_legend_show=False,
        )

pie = Pie("投资比例", title_pos="55%")
pie.add(
    "",
    attr_vip,
    value_vip,
    radius=[45, 65],
    center=[65, 50],
    legend_pos="80%",
    legend_orient="horizontal",
    is_label_show=False
)

grid = Grid(width=900)
grid.add(bar, grid_right="50%")
grid.add(pie)
grid

### 4、本月券使用情况

In [17]:
df_quan['券名称'] = df_quan['券别名'].apply(lambda x:re.sub(r'（\d+）$', "", x))

gp_quan = df_quan.groupby(['券名称','大类'])['投标金额'].sum() 

df_quan_res = gp_quan.unstack().dropna(subset=['加息券', '抵用券'],how='all')

df_quan_res.fillna(0,inplace=True)

attr = df_quan_res.index
v1 = np.around(df_quan_res['加息券'] / 10000)
v2 = np.around(df_quan_res['抵用券'] / 10000)

bar = Bar("券投资额","按券名称统计",width=900)
bar.add('加息券',attr,v1,mark_line=["average"],yaxis_formatter=" 万元",xaxis_rotate=-45)
bar.add('抵用券',attr,v2,mark_line=["average"],yaxis_formatter=" 万元",xaxis_rotate=-45)
bar

# 三、回款统计

In [18]:
attr_hk = pd.to_datetime(df_huikuan['回款时间']).dt.strftime('%m-%d')
value_jd_hk = np.around(df_huikuan['经典版'] / 10000)
value_cg_hk = np.around(df_huikuan['存管版'] / 10000)

bar = Bar("未来30天回款金额统计",width=900)
bar.add("经典版", attr_hk, value_jd_hk, is_stack=True,yaxis_formatter=" 万元")
bar.add("存管版", attr_hk, value_cg_hk, is_stack=True,yaxis_formatter=" 万元")

bar